In [1]:
import numpy as np
from embedding_manager import EmbeddingManager

In [2]:
em = EmbeddingManager(path='../fasttext.wiki-news-300d-1M.vec')

100%|██████████| 999995/999995 [01:17<00:00, 12963.95it/s]


Total embeddings shape: (629762, 300)


In [3]:
power = em.get_vector('power')
authority = em.get_vector('authority')
government = em.get_vector('government')
man = em.get_vector('man')

avg_vector = np.mean([power, authority, government, man], axis=0)
em.get_words(avg_vector, 10)

Estimator is being prepared...


[('authority', 1.0289617947526117),
 ('power', 1.1381789353724092),
 ('government', 1.1628852017446072),
 ('governmental', 1.1726115075890766),
 ('man', 1.1744869205508266),
 ('ruler', 1.2175793611453776),
 ('statesman', 1.21860621679381),
 ('woman', 1.2313762892635538),
 ('dominion', 1.2320704318769573),
 ('machinery', 1.2456221388303736)]

Good enough. Embeddings do not show that kind of insight as they show in typical embeddings tutorial, but it is as expected.

There is an option to reduce dimensionality either by taking other embeddings or through PCA, and this might get a little more robust results.